In [62]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.impute import KNNImputer
from scipy.stats import stats
from sklearn.decomposition import PCA
from sklearn.metrics import mean_absolute_error,r2_score,mean_squared_error

In [63]:
df=pd.read_csv("C:\\Users\\IBRAHIM KHAN\\Desktop\\FYP2024\\Final Year Project\\student_data.csv")

In [64]:
df

,Gender,Age,Address,Family_size,Mother_Edu,Father_Edu,Mother_job,Father_job,Parent_Edu_level,Guardian,...,Study_Style,Parent_relationship_with_teacher,Study_enviroment,Learning_platforms,interest_in_field,major,number_of_abbsentes_month,quizzes_per_semester,time_managment_skill,GPA
0,Female,19.0,URBAN,8.0,High School,NaN,Yes,No,High School,Other,...,Individual,No,Average,NaN,Yes,BSC ELECTRICAL ENG,8.0,12.0,Good,2.48
1,Male,NaN,URBAN,8.0,NaN,Master,Yes,Yes,Master,NaN,...,Individual,No,Poor,Platform4,No,BSC ELECTRICAL ENG,10.0,8.0,Poor,2.20
2,Female,19.0,RURAL,4.0,NaN,PhD,Yes,NaN,NaN,Mother,...,Individual,Yes,Average,Platform4,Yes,BS MLT,7.0,3.0,Average,2.93
3,Female,23.0,RURAL,13.0,High School,PhD,No,Yes,High School,Other,...,Individual,No,Poor,Platform1,No,BE CIVIL,10.0,10.0,Average,2.44
4,Female,19.0,RURAL,3.0,Master,Master,No,No,Bachelor,Other,...,Group,No,Good,Platform3,Yes,NaN,2.0,14.0,Average,2.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,NaN,21.0,NaN,NaN,NaN,Bachelor,Yes,No,PhD,Father,...,Individual,No,Poor,Platform1,NaN,BE CIVIL,5.0,11.0,Average,3.26
396,NaN,23.0,URBAN,3.0,NaN,NaN,Yes,Yes,Master,Father,...,Group,No,Poor,Platform5,Yes,BSC ELECTRICAL ENG,7.0,11.0,Poor,2.96
397,Female,18.0,URBAN,2.0,NaN,Master,Yes,No,PhD,Father,...,Individual,No,Average,Platform3,No,BS CARDIOLOGY,10.0,1.0,Poor,2.59
398,Male,21.0,RURAL,3.0,PhD,NaN,No,No,Bachelor,Mother,...,NaN,Yes,Average,Platform5,NaN,BS COMPUTER SCIENCE,3.0,0.0,Good,2.43


In [65]:
drop_col=df.drop(columns=['Age','Address','number_of_abbsentes_month','Gender','Mother_Edu','Father_Edu','Mother_job','Father_job','Study_Style','Learning_platforms','Study_enviroment','major','time_managment_skill'])

In [66]:
drop_col['GPA']=drop_col['GPA'].fillna(drop_col['GPA'].median())

In [67]:
X_train,X_test,y_train,y_test=train_test_split(drop_col.drop(columns=['GPA']),drop_col['GPA'],test_size=0.2,random_state=42)

In [68]:
for index, column in enumerate(X_train.columns):
    print(f"Index {index}: {column}")

Index 0: Family_size
Index 1: Parent_Edu_level
Index 2: Guardian
Index 3: Travel_time
Index 4: Std_hr_per_week
Index 5: Attendance
Index 6: Activities
Index 7: Internet_access
Index 8: Family_support
Index 9: Extra_tutor
Index 10: Parent_relationship_with_teacher
Index 11: interest_in_field
Index 12: quizzes_per_semester


In [69]:
categorical_features=[1,2,6,7,8,9,10,11]
knn_impute=[0,3,4,5,12]

In [70]:
mode_transformer=Pipeline(steps=[
    ('mode_impute',SimpleImputer(strategy='most_frequent')),
    ('ohe_mode',OneHotEncoder(drop='first',sparse_output=False,handle_unknown='ignore')),
    #('poly', PolynomialFeatures(degree=2, include_bias=False)),

])
KNN_transformer=Pipeline(steps=[
     ('knn_impute',KNNImputer(n_neighbors=19)),
    #('outlier_trans',OutlierCap(columns=knn_impute)),
    #('function_trans1',FunctionTransformer(func=np.log1p)),
    #because the data is already or almost normal distributed  when we use this transformer it's accuracy will decrease 
    # because i think data will shift left or right skew data
    ('power_tran',PowerTransformer()),
    ('scale_knn',StandardScaler()),
    #('poly', PolynomialFeatures(degree=2, include_bias=False)),
])

processor=ColumnTransformer([
    ('categorical_impute',mode_transformer,categorical_features),
    
    ('KNN_impute',KNN_transformer,knn_impute),
   # ('knn_attendance',knn_attendance_trans,knn_attendence),
    #('arbitrary_impute',arbitrary_transformer,arbitrary_impute)
])
lr=LinearRegression()
pipe=Pipeline([
    ('processor', processor),
    #('pca', PCA(n_components=0.98)),
    ('lr',lr)
])


In [71]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('processor',
                 ColumnTransformer(transformers=[('categorical_impute',
                                                  Pipeline(steps=[('mode_impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('ohe_mode',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  [1, 2, 6, 7, 8, 9, 10, 11]),
                                                 ('KNN_impute',
                                                  Pipeline(steps=[('knn_impute',
                                                                   KNNImputer(n_neighbors=19)),
                                                                  ('power_tran',
                                                                   PowerTransformer()),
                                                                  ('scale_knn',
                                                                   StandardScaler())]),
                                                  [0, 3, 4, 5, 12])])),
                ('lr', LinearRegression())])

In [72]:
y_pred=pipe.predict(X_test)
y_pred

array([2.75000892, 2.10463125, 2.5753721 , 2.07354976, 1.85725405,
       2.95189964, 1.70920473, 2.82198669, 2.77512951, 2.78086928,
       2.09806679, 2.7905466 , 2.89120437, 2.84403026, 2.86379426,
       2.94661642, 2.58518307, 3.0766146 , 2.11124374, 1.93651923,
       2.52001182, 2.31220552, 2.84552159, 1.70645302, 2.39594688,
       2.83523863, 2.57140405, 3.02027653, 2.7473444 , 2.6687757 ,
       3.0334778 , 2.22077208, 1.90351542, 1.92984981, 3.60020463,
       2.74230495, 2.98191099, 2.03562589, 2.80436106, 3.02425538,
       3.31236896, 1.69276095, 2.26042998, 2.65042975, 2.51183327,
       2.61950929, 2.51724836, 2.25227967, 3.38130534, 2.70767239,
       2.74622893, 2.7649924 , 2.70167623, 2.73620314, 2.07108405,
       1.80271961, 1.90483333, 3.13963997, 1.96085183, 2.8617396 ,
       2.582803  , 1.90537324, 3.39085174, 3.00401255, 2.34739627,
       2.72965413, 1.9581343 , 2.01032624, 2.95047242, 1.7516761 ,
       3.3299931 , 2.82040965, 2.38800591, 2.34376122, 2.80458

In [73]:
print("MAE" ,mean_absolute_error(y_test,y_pred))
print("MSE",mean_squared_error(y_test,y_pred))
print("r2_score", r2_score(y_test,y_pred))

MAE 0.19262003132809097
MSE 0.05142295566337285
r2_score 0.8122771922291429


In [75]:
import joblib
joblib.dump(pipe,"linear_regression.joblib")

['linear_regression.joblib']